In [231]:
# Run once to install necessary packages
!pip install pandas numpy matplotlib seaborn scikit-learn openpyxl


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [232]:
import pandas as pd
import numpy as np


# Analisis  general

Se procede a analizar una por una las tablas asociadas al Excel que se envia. Luego se procede a crear los respectivos DF asociados como CSV file en caso de requerer procesamiento y seprar tablas para su respectivo manejo en el proceso de data wrangling

In [ ]:
# Primer tablab
detalle_cliente = pd.read_excel("../data/bronze/Prueba proceso de selección Analista de Datos - bases.xlsx", 
                                sheet_name="Detalle_cliente")
print(detalle_cliente.shape)
print("=="*100)
print(detalle_cliente.dtypes)
print("=="*100)
detalle_cliente.head(3)

(2500, 3)
Id_cliente          int64
fecha_efectiva    float64
Id_tx               int64
dtype: object


,Id_cliente,fecha_efectiva,Id_tx
0,136,20210303.0,6869
1,116,20201129.0,4355
2,119,20200929.0,5416


In [ ]:
# Segunda tabla
detalle_tx = pd.read_excel("../data/bronze/Prueba proceso de selección Analista de Datos - bases.xlsx",
                           sheet_name = "Detalle_tx" )
print(detalle_tx.shape)
print("=="*100)
print(detalle_tx.dtypes)
print("=="*100)
detalle_tx.head(3)


(2500, 3)
Id_tx     int64
clase    object
valor    object
dtype: object


,Id_tx,clase,valor
0,6869,COMPRA,3495240
1,4355,COMPRA,475800
2,5416,COMPRA,115775


# Reto

1) Encuentre un habito transacional de los cuentes o patrones de comportamiento. Haga un descriptivo
2) Identifique grupos para cada estrategia, justifique una emtodologia que le permita usar algun criterio y mensione numero de clientes por grupo.

**Consideraciones**

 a. La estrategia consiste en designar descuentos del 5,20,25% en su proxima compra a los clientes. *Priorizar desceutnos en clientes con menos uso de tarjetas (compras y avances)*

 b. Clientes afines avances, no compras. *Incentivar compras*

 c. Cliente mas afin: Clientes con mayor numero de compras, facturacion y utlizacion en pocos dias.



# Solucion propuesta

Se realizan 3 analisis importantes. 

1) Calidad de datos: Para identificar que hay, de que forma es, si hay errores.
2) Analisis descrptivo: Como son los datos? Tienen algun patron visible a simple vista?
3) Modelado: Tentativamente, problema no supervisado donde se podrian buscar cluster en los datos despues de las respectivas tecnicas de escalado y preprocesado para modelar.

Todo este proceso es tangencial a la idea del negocio adyacente. Toda decision basado en los datos requerirá ser verificada desde la logica del neogcio para evitar absurdos.

Se usaran notebooks para hacer analisis exploratorios a la informacion pero eventualmente, la modularizacion de un pipeline completo será requerida. So, let's go!

# Detalles clientes


In [235]:
# Analisis primer tabla - detalle_cliente
detalle_cliente.info() # Hay null values en la fecha

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id_cliente      2500 non-null   int64  
 1   fecha_efectiva  2488 non-null   float64
 2   Id_tx           2500 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 58.7 KB


In [236]:
print("TOTAL CLIENTES UNICOS:",detalle_cliente["Id_cliente"].nunique()) # Existen en total 70 clientes en el dataset
detalle_cliente[detalle_cliente["fecha_efectiva"].isna()]

TOTAL CLIENTES UNICOS: 70


,Id_cliente,fecha_efectiva,Id_tx
290,113,NaN,9858
562,119,NaN,5193
779,112,NaN,4647
987,108,NaN,4081
1179,121,NaN,5994
1371,148,NaN,9035
1579,99,NaN,5939
1779,102,NaN,9792
1946,123,NaN,6826
2035,137,NaN,8655


In [237]:
detalle_cliente["fecha_efectiva"] = pd.to_datetime(
    detalle_cliente["fecha_efectiva"].astype("Int64"),  # cambianbdo fecha a formato trabhajble
    format="%Y%m%d",
    errors="coerce"
)

print(detalle_cliente.info())
print(detalle_cliente.head(3))

ValueError: 'indices' contains values less than allowed (-119 < -1)

``Inisghts``
 1) Existen 12 registros sin el campo fecha efectiva. Se propone borrarlos puesto que esto no es recuperable para ningun otro registro (no hay forma de mapear esto con la otra tabla auxiliar)
 2) Esta tabla tiene informacion relevante del precio y fecha para cada cliente pero mas importante **tiene un indicador para ser unida con la segunda tabla**
 3) La columna de fecha ha de ser investigada a profundidad para tratarla correctamente

# Detalle tx

In [ ]:
print(detalle_tx.info()) # Clase y valor algunos poseen valores nulos
print("=="*100)
print(detalle_tx["clase"].value_counts()) # Ver niveles de la variable categorica. Clases desbalanceadas. 
print("=="*100)
print(detalle_tx["valor"].describe()) # Ver estadisticas de la variable numerica

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id_tx   2500 non-null   int64 
 1   clase   2495 non-null   object
 2   valor   2476 non-null   object
dtypes: int64(1), object(2)
memory usage: 58.7+ KB
None
clase
COMPRA    1931
AVANCE     558
SEGURO       6
Name: count, dtype: int64
count       2476
unique      1720
top       800000
freq         136
Name: valor, dtype: int64


In [ ]:
detalle_tx[detalle_tx["clase"].isna() | detalle_tx["valor"].isna()] # Ver registros con valores nulos

,Id_tx,clase,valor
281,6890,COMPRA,NaN
635,4855,COMPRA,NaN
943,2847,COMPRA,NaN
1089,2773,COMPRA,NaN
1093,3334,COMPRA,NaN
1179,5994,COMPRA,NaN
1233,7801,COMPRA,NaN
1253,8438,COMPRA,NaN
1386,4065,COMPRA,NaN
1455,6358,AVANCE,NaN


In [ ]:
# Inputando correctamente valor
detalle_tx["valor"] = detalle_tx.valor.astype(float) # Cambiando tipo de dato a float

ValueError: could not convert string to float: '41f30'

``insigths``
 1) Hay valores nulos en clase y valor. Se procede a eliminar estos registros dado que no poseen informacion que sea capturable en el estadoa ctual de la informacion. 
 2) Existen formatos no numericos en el campo valor. Se procede a mirar aquellos valroes que no sean numericos con regrex para irar si son errores de inputacion y se pueden corregir o si simplemente se ha de eliminar aquello con mal formato


# Data cleaning - Detalles cliente

## Correcion fecha

In [ ]:
 # Eliminando registros sin fecha efectiva
detalle_cliente = detalle_cliente.dropna(subset=["fecha_efectiva"])
# Se sabe que el formato por defecto de fecha es YYYYMMDD, por lo que se puede transformar usando regrex
detalle_cliente["fecha_efectiva"] = detalle_cliente["fecha_efectiva"].astype(float).astype("Int64").astype(str)


# Fecha como '20210303'
pattern = r"^\d{8}$"  

# Mostrar registros no validos
print("Registros con formato de fecha no válido:")
print(detalle_cliente[~detalle_cliente["fecha_efectiva"].str.match(pattern, na=False)])

# Correcion fecha
detalle_cliente = detalle_cliente[detalle_cliente["fecha_efectiva"].str.match(pattern, na=False)]

Registros con formato de fecha no válido:
      Id_cliente fecha_efectiva  Id_tx
1287         118         202012   9186
1624         123         202001   4160
1869         131           2021   5464
2148         112           2020   4500


In [ ]:
detalle_cliente.info() # Todo ok

<class 'pandas.core.frame.DataFrame'>
Index: 2484 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id_cliente      2484 non-null   int64 
 1   fecha_efectiva  2484 non-null   object
 2   Id_tx           2484 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 77.6+ KB


# Data cleaning - detalle tx

## Correcion clase

In [ ]:
detalle_tx = detalle_tx[detalle_tx["clase"].notna() & detalle_tx["valor"].notna()] # Eliminando registros con valores nulos
detalle_tx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2471 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id_tx   2471 non-null   int64 
 1   clase   2471 non-null   object
 2   valor   2471 non-null   object
dtypes: int64(1), object(2)
memory usage: 77.2+ KB


In [ ]:
detalle_tx.clase.value_counts() # Todo Ok con clase

clase
COMPRA    1911
AVANCE     554
SEGURO       6
Name: count, dtype: int64

## Correcion valor

In [249]:
detalle_tx.head() # En genreral, data es int

,Id_tx,clase,valor
0,6869,COMPRA,3495240
1,4355,COMPRA,475800
2,5416,COMPRA,115775
3,3249,COMPRA,1579898
4,2774,COMPRA,84077


In [ ]:
import re
import numpy as np

pattern = r'\d+(\.\d+)?' # Expresión regular para números enteros o decimales
# Filtrar filas con valores mal formateados
detalle_tx[~detalle_tx["valor"].astype(str).str.fullmatch(pattern, na=True)]


,Id_tx,clase,valor
281,6890,COMPRA,NaN
635,4855,COMPRA,NaN
797,9042,COMPRA,41f30
943,2847,COMPRA,NaN
1089,2773,COMPRA,NaN
1093,3334,COMPRA,NaN
1179,5994,COMPRA,NaN
1233,7801,COMPRA,NaN
1253,8438,COMPRA,NaN
1386,4065,COMPRA,NaN


In [251]:
#detalle_tx["valor"] = detalle_tx.valor.astype(float) # hay un error con los valores de esta colubna
# Se elimnaan na y valores mal imputads
def es_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
# funcion lambda para convertir a numerico o NaN
detalle_tx["valor"] = detalle_tx["valor"].apply(lambda x: float(x) if es_float(x) else np.nan)
detalle_tx['valor'].isna().sum() # Se encojntraron 26 datos mal inputados


np.int64(26)

In [254]:
detalle_tx.dropna(subset=["valor"], inplace=True) # Eliminando registros con valores nulos

# Uniendo DF

Claramente esta iunformacion fue diseñada para unir los dataframes. Vamos a crear la Master Table una vez los datos se han filtrado, para realizar analisis mas completos


In [260]:

detalle_cliente["fecha_efectiva"] = pd.to_datetime(detalle_cliente["fecha_efectiva"], format="%Y%m%d", errors="coerce")
detalle_cliente.head()

,Id_cliente,fecha_efectiva,Id_tx
0,136,2021-03-03,6869
1,116,2020-11-29,4355
2,119,2020-09-29,5416
3,121,2021-02-22,3249
4,152,2020-09-30,2774


In [257]:
df = detalle_cliente.merge(detalle_tx, on = "Id_tx", how="left")
df.head()

,Id_cliente,fecha_efectiva,Id_tx,clase,valor
0,136,20210303.0,6869,COMPRA,3495240.00
1,116,20201129.0,4355,COMPRA,475800.00
2,116,20201129.0,4355,COMPRA,350503.62
3,119,20200929.0,5416,COMPRA,115775.00
4,121,20210222.0,3249,COMPRA,1579898.00


In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id_cliente      3321 non-null   int64  
 1   fecha_efectiva  3304 non-null   float64
 2   Id_tx           3321 non-null   int64  
 3   clase           3297 non-null   object 
 4   valor           3302 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 129.9+ KB
